In [2]:
import sqlite3

In [3]:
sqlite_file = 'insurance.sqlite'

In [7]:
conn = sqlite3.connect(sqlite_file)
cur = conn.cursor()
SQL = "select * from complete"
cur.execute(SQL)

In [8]:
#convert the table "complete" to pandas dataframe 
import pandas as pd

data = cur.fetchall()

colnames = [desc[0] for desc in cur.description]

df = pd.DataFrame(data)
df.columns = colnames

In [9]:
df

,CONTRACT_ID,CUSTOMER_BIRTHDATE,CUSTOMER_GENDER,CUSTOMER_CITY,CAR_IDENT,CONTRACT_DATE,CONTRACT_CLASS,INSURANCE_CLASS,MAX_REFUND,POPULATION,MAKE,MODEL,DISPLACEMENT,POWER,LUXURY
0,1,1985-03-28,M,rome,meE200,2016-07-15,1,1,6000000,27060000,mercedes,E200 CDI,2.15,100,Y
1,2,1975-06-12,M,florence,opCR17D,2016-01-20,10,10,2500000,366000,opel,Corsa 1.7 CDTI,1.70,75,N
2,3,1987-10-03,F,milan,fi500b14,2017-08-01,14,14,2000000,1296000,fiat,Fiat 500 1.4,1.40,60,N


In [10]:
#calculate base price
import numpy as np

df['BASE_PRICE'] = 400.0 + 20.0 * df['DISPLACEMENT'] + \
                    30.0 * np.log2(df['MAX_REFUND']) - \
                    ((15.0 - df['INSURANCE_CLASS'])**2)

In [12]:
df

,CONTRACT_ID,CUSTOMER_BIRTHDATE,CUSTOMER_GENDER,CUSTOMER_CITY,CAR_IDENT,CONTRACT_DATE,CONTRACT_CLASS,INSURANCE_CLASS,MAX_REFUND,POPULATION,MAKE,MODEL,DISPLACEMENT,POWER,LUXURY,BASE_PRICE
0,1,1985-03-28,M,rome,meE200,2016-07-15,1,1,6000000,27060000,mercedes,E200 CDI,2.15,100,Y,922.495932
1,2,1975-06-12,M,florence,opCR17D,2016-01-20,10,10,2500000,366000,opel,Corsa 1.7 CDTI,1.70,75,N,1046.604900
2,3,1987-10-03,F,milan,fi500b14,2017-08-01,14,14,2000000,1296000,fiat,Fiat 500 1.4,1.40,60,N,1054.947057


In [13]:
#calculate the new customer flag
import datetime as dt

print(pd.to_datetime('now').date())

df['NEW_CUSTOMER'] = [True if days < 30 else False for days
                     in(pd.to_datetime('now').date() - \
                        pd.to_datetime(df['CONTRACT_DATE']).dt.date).dt.days]

2020-06-29


In [16]:
#calculte the driver age
df['DRIVER_AGE'] = (pd.to_datetime('now').date() - \
                    pd.to_datetime(df['CUSTOMER_BIRTHDATE']).dt.date).astype('m8[Y]')

In [17]:
df

,CONTRACT_ID,CUSTOMER_BIRTHDATE,CUSTOMER_GENDER,CUSTOMER_CITY,CAR_IDENT,CONTRACT_DATE,CONTRACT_CLASS,INSURANCE_CLASS,MAX_REFUND,POPULATION,MAKE,MODEL,DISPLACEMENT,POWER,LUXURY,BASE_PRICE,NEW_CUSTOMER,DRIVER_AGE
0,1,1985-03-28,M,rome,meE200,2016-07-15,1,1,6000000,27060000,mercedes,E200 CDI,2.15,100,Y,922.495932,False,35.0
1,2,1975-06-12,M,florence,opCR17D,2016-01-20,10,10,2500000,366000,opel,Corsa 1.7 CDTI,1.70,75,N,1046.604900,False,45.0
2,3,1987-10-03,F,milan,fi500b14,2017-08-01,14,14,2000000,1296000,fiat,Fiat 500 1.4,1.40,60,N,1054.947057,False,32.0


In [18]:
#recode some variables
df['CUSTOMER_GENDER'] = ['MALE' if g == 'M' else 'FEMALE' for g in df['CUSTOMER_GENDER']]
df['LUXURY'] = df['LUXURY'] == 'Y'

In [19]:
df

,CONTRACT_ID,CUSTOMER_BIRTHDATE,CUSTOMER_GENDER,CUSTOMER_CITY,CAR_IDENT,CONTRACT_DATE,CONTRACT_CLASS,INSURANCE_CLASS,MAX_REFUND,POPULATION,MAKE,MODEL,DISPLACEMENT,POWER,LUXURY,BASE_PRICE,NEW_CUSTOMER,DRIVER_AGE
0,1,1985-03-28,MALE,rome,meE200,2016-07-15,1,1,6000000,27060000,mercedes,E200 CDI,2.15,100,True,922.495932,False,35.0
1,2,1975-06-12,MALE,florence,opCR17D,2016-01-20,10,10,2500000,366000,opel,Corsa 1.7 CDTI,1.70,75,False,1046.604900,False,45.0
2,3,1987-10-03,FEMALE,milan,fi500b14,2017-08-01,14,14,2000000,1296000,fiat,Fiat 500 1.4,1.40,60,False,1054.947057,False,32.0


In [20]:
#loop through the df
#format each row in the following format,
#and export it to txt file ("facts.txt") as the input to CLIPS
#
#(contract
#    (contract-id 1)
#    (car-luxury TRUE)
#    (car-power 100)
#    (car-displacement 2.15)
#    (new-customer FALSE)
#    (contract-class 1)
#    (max-refund 6000000)
#    (driver-age 32)
#    (driver-gender MALE)
#    (driver-city-population 2706000)
#    (base-price 922.495932)
#    )

In [ ]:
fout = open()